In [ ]:
%load_ext autoreload
%autoreload 2
import requests

import requests
import json

# base url
base_url = "http://api:5000/istsos4/v1.1/"

# Headers (optional, but specifying Content-Type ensures proper handling of JSON data)
headers = {
    'Content-Type': 'application/json'
}

### Creation Order
The SensorThingsAPI data schema has relations.  
So to respect references among elements you have to follow a defined order in creating them.  
The usual order is:

1. Thing
2. Location
3. ObservedProperty
4. Sensor
5. Datastream
6. Observation (+FeatureOfInterest)


### Thing creation

In [ ]:
body = {
  "name" : "MAG_LOD4",
  "description" : "Maggia river in Lodrino",
  "properties" : {
    "subcatchment" : "Maggia",
    "subcatchment_area" : 4000,
    "subcatchment_number": 12345
  }
}

# POST request with the JSON body
response = requests.post(base_url + 'Things', data=json.dumps(body), headers=headers)

In [ ]:
# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print("Thing created successfully.")
    print(response.headers['location'])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [ ]:
# Let's get the Things @iot.id usimng a regex to extract the number in parentheses
import re
match = re.search(r'\((\d+)\)', response.headers['location'])

if match:
    things_id = int(match.group(1))
    print(f"Number extracted: {things_id}")
else:
    print("No number found in parentheses.")

### Create an associated Location

In [ ]:
body = {
  "name": "Location of the MAG_LOD gauge",
  "description": "The gauge is located close to the bridge",
  "properties": {},
  "encodingType": "application/geo+json",
  "location": {
    "type": "Point",
    "coordinates": [8.10, 50.00]
  },
  "Things": [
    { "@iot.id": things_id}
  ]
}

print(json.dumps(body))

# POST request with the JSON body
response = requests.post(base_url + 'Locations', data=json.dumps(body), headers=headers)

In [ ]:
# Check if the request was successful (status code 2xx)
if response.status_code == 201:
    print("Location created successfully.")
    print(response.headers['location'])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [ ]:
Things = meteo.query_api('Things','$expand=Locations')

In [ ]:
map = meteo.map_things(Things)

In [ ]:
map